# Initialization

Test notebook for the C-MAPPS benchmark. Test RNN architecture. Tensorflow implementation 

First we import the necessary packages and create the global variables.

In [1]:
import math
import numpy as np
import csv
import copy
import tensorflow as tf
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


# Define the Model

Define the RNN model using tensorflow

In [2]:
lambda_regularization = 0.20

def RNNModel(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    model.add(LSTM(input_shape=input_shape))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model
    

def RULmodel_SN_6(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(10, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Create tunable model

Create the tunable model to train the data

In [3]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

#Selected from the results of running DE on the old model (250->50->1)
windowSize = 1
stride = 1
constRUL = 140

dataFolder = '../CMAPSSData'

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_SN_6(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_SN_1', modelRULSN, selected_features, dataFolder, 'keras', sequenceModel=True, 
                      window_size = windowSize, scaler = min_max_scaler, window_stride = stride, constRul = constRUL)
#tModel.sequenceLength = 0
tModel.dataScaler = None
#tModel.sequenceStrategy = 'avg/bottom'
tModel.changeDataset(1)
tModel.loadData(verbose=1, rectify_labels = False)
tModel.printData()

Loading data for dataset 1 with window_size of 1, stride of 1 and constRUL of 140. Cros-Validation ratio 0
True
Using average sequence length 206
(206, 14)
(206,)
[[  641.82    1589.7     1400.6    ...,   392.        39.06      23.419 ]
 [  641.82    1589.7     1400.6    ...,   392.        39.06      23.419 ]
 [  641.82    1589.7     1400.6    ...,   392.        39.06      23.419 ]
 ..., 
 [  643.64    1599.22    1425.95   ...,   398.        38.49      23.0675]
 [  643.34    1602.36    1425.77   ...,   394.        38.45      23.1295]
 [  643.54    1601.41    1427.2    ...,   396.        38.48      22.9649]]
[ 140.  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.
  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.
  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.
  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.
  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.  140.
  140.  140.  140.  

In [13]:
a = np.ones((5))
print(a)
b = np.tile(a, (5,1))
print(b)

[ 1.  1.  1.  1.  1.]
[[ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.]]
